In [ ]:
!rm -rf /content/drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Install dependencies
from google.colab import files

!pip install opencv-python-headless tensorflow
!pip install mediapipe opencv-python
!pip install gradio
!pip install rembg onnxruntime
# First, install the rembg library if you haven't already
!pip install rembg
!sudo apt update && sudo apt install ffmpeg
!pip install moviepy
!pip install -r /content/drive/MyDrive/agebased_motion_application_deeplearning_project/requirements.txt
!pip install sync-batchnorm
!pip install opencv-python-headless tensorflow
!pip install ffmpeg-python
!pip install streamlit

In [ ]:
#Integrated module
import cv2 as cv
import sys
sys.path.insert(0, '/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modules') # Add this line with the correct path to demo.py
from demo import load_checkpoints, make_animation
sys.path.append('/content/drive/MyDrive/agebased_motion_application_deeplearning_project')
sys.path.append('/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modules')
import PIL.Image
import ffmpeg
import imageio
import numpy as np
import os
import warnings
from base64 import b64encode
from skimage import img_as_ubyte
from tempfile import NamedTemporaryFile
from tqdm.auto import tqdm
import mediapipe as mp
import shutil
from moviepy.editor import VideoFileClip, clips_array
import gradio as gr

def agespecificmotion(img1, img2, img3):
    """
    A single function to handle the entire pipeline for generating an age-specific animation.
    Takes three uploaded image paths as input.
    """
    # Save uploaded images to specific paths
    saved_paths = [
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image1.png",
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image2.png",
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image3.png"
    ]
    # Save images to predefined paths
    for img, path in zip([img1, img2, img3], saved_paths):
        import shutil
        shutil.copy(img, path)

    # pipeline code (modules/functions)
    # Age and Gender Detection Module
    def getFaceBox(net, frame, conf_threshold=0.7):
        frameHeight = frame.shape[0]
        frameWidth = frame.shape[1]
        blob = cv.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], True, False)

        net.setInput(blob)
        detections = net.forward()
        bboxes = []
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > conf_threshold:
                x1 = int(detections[0, 0, i, 3] * frameWidth)
                y1 = int(detections[0, 0, i, 4] * frameHeight)
                x2 = int(detections[0, 0, i, 5] * frameWidth)
                y2 = int(detections[0, 0, i, 6] * frameHeight)
                bboxes.append([x1, y1, x2, y2])
        return bboxes

    faceProto = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/opencv_face_detector.pbtxt"
    faceModel = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/opencv_face_detector_uint8.pb"
    ageProto = '/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/age_deploy.prototxt'
    ageModel = '/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/age_net.caffemodel'
    genderProto = '/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/gender_deploy.prototxt'
    genderModel = '/content/drive/MyDrive/agebased_motion_application_deeplearning_project/modelNweight/gender_net.caffemodel'

    MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
    ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-55)', '(56-70)', '(71-100)']
    genderList = ['Male', 'Female']

    # Load network
    ageNet = cv.dnn.readNet(ageModel, ageProto)
    genderNet = cv.dnn.readNet(genderModel, genderProto)
    faceNet = cv.dnn.readNet(faceModel, faceProto)

    padding = 20

    def age_gender_detector(frame):
        bboxes = getFaceBox(faceNet, frame)
        ages = []
        for bbox in bboxes:
            face = frame[max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
                        max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

            blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
            genderNet.setInput(blob)
            genderPreds = genderNet.forward()
            ageNet.setInput(blob)
            agePreds = ageNet.forward()

            # Get the detected age range
            age_range = ageList[agePreds[0].argmax()]  # Get the age category

            # Convert age range to midpoint integer (used for the next module)
            age_min, age_max = map(int, age_range.strip('()').split('-'))
            age_mid = (age_min + age_max) // 2
            ages.append(age_mid)  # Store the midpoint integer

        return ages  # Return midpoints for the next module

    # List of input image paths (change the paths to your images)
    image_paths = [
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image1.png",
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image2.png",
        "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/inputimages/image3.png"
    ]

    # Variables to store detected ages for each image
    detected_ages = []  # To hold ages for all images
    all_age_categories = []
    for idx, image_path in enumerate(image_paths):
        input_image = cv.imread(image_path)
        ages = age_gender_detector(input_image)  # Get detected ages (midpoints) for the current image
        detected_ages.append(ages)  # Append to the detected_ages list

    # Output the detected age categories for the print statement (just categories, not exact ages)
    for idx, age in enumerate(detected_ages):
        # Get the age categories for printing based on midpoint
        age_categories = []
        for mid_age in age:
            # Match the midpoint to its category
            for category in ageList:
                age_min, age_max = map(int, category.strip('()').split('-'))
                if age_min <= mid_age <= age_max:
                    age_categories.append(category)
                    break
        all_age_categories.append(age_categories)
        print(f"Detected Age Categories for Image {idx + 1}: {', '.join(age_categories)}")
    age_message = (f"The person in the first picture belongs to the age category {all_age_categories[0][0]}, "
               f"the person in the second picture belongs to the age category {all_age_categories[1][0]}, "
               f"and the person in the third picture belongs to the age category {all_age_categories[2][0]}.")

    # First order motion model Module

    warnings.filterwarnings("ignore")

    age_motion_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/agemotionvideos"
    output_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/output_videos"

    # Ensure output directory exists
    os.makedirs(output_videos_dir, exist_ok=True)

    # Function to get the driving video path based on the age, alternating videos if multiple inputs fall in the same category
    category_counter = {
      '7-12': 0,
      '21-37': 0,
      '13-20':0
    }

    def get_driving_video_path(age):
      if age < 2:
          return os.path.join(age_motion_videos_dir, '0to2.mp4')
      elif 3 <= age < 6:
          return os.path.join(age_motion_videos_dir, '4to6.mp4')
      elif 7 <= age < 12:
          video_file = f'8to12v{category_counter["7-12"] % 2 + 1}.mp4'
          category_counter['7-12'] += 1
          return os.path.join(age_motion_videos_dir, video_file)
      elif 13 <= age < 20:
          video_file = f'13to20v{category_counter["13-20"] % 2 + 1}.mp4'
          category_counter['13-20'] += 1
          return os.path.join(age_motion_videos_dir, video_file)

      elif 21 <= age < 37:
          video_file = f'21to37v{category_counter["21-37"] % 2 + 1}.mp4'
          category_counter['21-37'] += 1
          return os.path.join(age_motion_videos_dir, video_file)
      elif 38 <= age < 55:
          return os.path.join(age_motion_videos_dir, '38to55.mp4')
      elif 56 <= age < 70:
          return os.path.join(age_motion_videos_dir, '56to70.mp4')
      elif 71 <= age < 100:
          return os.path.join(age_motion_videos_dir, '71to100.mp4')

    # Function to resize images to (256, 256)
    def resize_image(image):
      return image.resize((256, 256), resample=PIL.Image.LANCZOS)

    # FOMM processing function
    def fomm_process(image_path, driving_video_path, output_dir):
      # Load and resize the input image
      input_image = PIL.Image.open(image_path).convert("RGB")
      input_image_resized = resize_image(input_image)

      # Load the driving video
      reader = imageio.get_reader(driving_video_path, mode='I', format='FFMPEG')
      fps = reader.get_meta_data()['fps']
      driving_video = [frame for frame in reader]

      # Load the FOMM model
      model_name = 'vox'  # Model name
      checkpoint_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/config/vox-256.yaml"
      weights_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/checkpoints/vox-cpk.pth"

      generator, kp_detector = load_checkpoints(config_path=checkpoint_path, checkpoint_path=weights_path)

      # Generate the animation
      predictions = make_animation(
          np.array(input_image_resized) / 255.0,  # Normalize the image
          [cv2.resize(frame, (256, 256)) / 255.0 for frame in driving_video],  # Normalize frames
          generator,
          kp_detector,
          relative=True,  # Adjust as necessary
          adapt_movement_scale=True  # Adjust as necessary
      )

      # Save the output video
      output_video_path = os.path.join(output_dir, os.path.basename(image_path).split('.')[0] + '_output.mp4')
      imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

      return output_video_path

    # Example usage
    detected_ages = []  # This will hold the ages from the age detection module

    # Assuming this part of the code is executed after running the age detection module
    for idx, image_path in enumerate(image_paths):
      input_image = cv.imread(image_path)
      ages = age_gender_detector(input_image)  # Get detected ages for the current image
      detected_ages.append(ages)  # Append to the detected_ages list

    output_videos = []
    for image_path, ages in zip(image_paths, detected_ages):
      # Use the first detected age for driving video selection
      driving_video_path = get_driving_video_path(ages[0])  # Use the first age in the list
      output_video = fomm_process(image_path, driving_video_path, output_videos_dir)
      output_videos.append(output_video)

    # Output video paths
    for idx, video in enumerate(output_videos):
      print(f"Output video for image {idx + 1}: {video}")



    # Background removal Module

    # Initialize MediaPipe variables
    mp_selfie_segmentation = mp.solutions.selfie_segmentation

    # Paths
    input_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/output_videos"
    bg_removed_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/bg_removed_videos"

    # Ensure output directory exists
    os.makedirs(bg_removed_videos_dir, exist_ok=True)

    # Function to remove background and save video with black background
    def remove_background(input_video_path, output_video_path):
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video file {input_video_path}")
            return

        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Use MP4 codec
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height), isColor=True)

        with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as selfie_segmentation:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert the frame to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = selfie_segmentation.process(rgb_frame)

                # Create the binary mask for segmentation
                condition = results.segmentation_mask > 0.5

                # Apply the mask to set background pixels to black
                output_frame = np.zeros_like(frame)  # Start with a black background
                output_frame[condition] = frame[condition]  # Set person pixels from original frame

                # Write the frame with the black background
                out.write(output_frame)

        cap.release()
        out.release()
        print(f"Background-removed video saved at {output_video_path}")

    # Process each video in the input directory
    for video_file in os.listdir(input_videos_dir):
        if video_file.endswith('.mp4'):
            input_video_path = os.path.join(input_videos_dir, video_file)
            output_video_path = os.path.join(bg_removed_videos_dir, video_file.replace('.mp4', '_no_bg.mp4'))
            remove_background(input_video_path, output_video_path)



    bg_removed_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/bg_removed_videos"
    bg_removed_videos = sorted([os.path.join(bg_removed_videos_dir, f) for f in os.listdir(bg_removed_videos_dir) if f.endswith('_no_bg.mp4')])

    def trim_videos(video_paths):
        # Load video clips and get their durations
        clips = []
        durations = []

        for path in video_paths:
            try:
                clip = VideoFileClip(path)
                clips.append(clip)
                durations.append(clip.duration)
            except Exception as e:
                print(f"Error loading video {path}: {e}")

        # Ensure we have valid clips to work with
        if not clips:
            print("No valid clips loaded for trimming.")
            return

        # Find the minimum duration in seconds
        min_duration = min(durations)
        print(f"Minimum duration found: {min_duration:.3f} seconds")

        # Trim each video to the minimum duration
        for i, clip in enumerate(clips):
            try:
                current_duration = clip.duration

                # Check if the current video needs trimming
                if current_duration > min_duration:
                    # Trim video to the minimum duration
                    trimmed_clip = clip.subclip(0, min_duration)

                    # Write the trimmed video back to the same file
                    trimmed_clip.write_videofile(video_paths[i], codec='libx264', audio_codec='aac', remove_temp=True)
                    trimmed_clip.close()  # Close the trimmed clip
                    print(f"Trimmed {video_paths[i]} to {min_duration:.3f} seconds")
                else:
                    print(f"{video_paths[i]} is already at or below the minimum duration.")

            except Exception as e:
                print(f"Error trimming video {video_paths[i]}: {e}")

        # Close all original clips
        for clip in clips:
            clip.close()



    # video concatenation Module

    # Paths
    bg_removed_videos_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/bg_removed_videos"
    merged_video_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/merged_video/merged_video.mp4"


    # Get all background-removed video files
    bg_removed_videos = sorted([os.path.join(bg_removed_videos_dir, f) for f in os.listdir(bg_removed_videos_dir) if f.endswith('.mp4')])

    # Load video clips
    clips = [VideoFileClip(video).resize((256, 256)) for video in bg_removed_videos]

    # Concatenate clips horizontally
    final_clip = clips_array([[clips[0], clips[1], clips[2]]])  # Adjust this if you have more or fewer clips

    # Write the output video
    final_clip.write_videofile(merged_video_path, codec='libx264', fps=30)

    print(f"Concatenated video saved at: {merged_video_path}")

    # Define paths
    merged_video_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/merged_video/merged_video.mp4"
    merged_video_frames_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/merged_video/merged_videoframes/"
    background_video_tracker = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/background_video_tracker.txt"
    background_video_dir = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/age_specific_motion/"
    background_frames_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/framesbackgroundvideo/"
    output_frames_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/outputvideoframes/"
    final_output_video_path = "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/final_outputvideo/final_output_video.mp4"


    # Utility: Clear frames from a folder without deleting the folder itself
    def clear_folder(folder_path):
        if os.path.exists(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove files or links
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove subdirectories
        else:
            os.makedirs(folder_path)  # Create the folder if it doesn't exist


    # Dynamic background video selection
    def load_background_video():
        # Read current tracker value
        with open(background_video_tracker, "r") as file:
            current_index = int(file.read().strip())

        # Update tracker to next video
        new_index = (current_index % 6) + 1
        with open(background_video_tracker, "w") as file:
            file.write(str(new_index))

        # Select background video path
        background_video_name = f"backgroundvideo{new_index}.mp4"
        background_video_path = os.path.join(background_video_dir, background_video_name)
        print(f"Selected Background Video: {background_video_path}")
        return background_video_path


    # Clear folders before processing
    clear_folder(merged_video_frames_path)
    clear_folder(background_frames_path)
    clear_folder(output_frames_path)

    # Step 1: Extract frames from a video
    def extract_frames_from_video(video_path, output_frames_path):
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_filename = os.path.join(output_frames_path, f'frame_{frame_count:04d}.png')
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        cap.release()
        print(f"Extracted {frame_count} frames from {video_path}.")
        return frame_count

    # Step 2: Replace black background with transparency
    def replace_black_background_with_transparency(frames_path):
        frame_files = sorted(os.listdir(frames_path))
        for frame_file in frame_files:
            frame_path = os.path.join(frames_path, frame_file)
            frame = cv2.imread(frame_path)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_black = np.array([0, 0, 0])
            upper_black = np.array([180, 255, 30])
            mask = cv2.inRange(hsv, lower_black, upper_black)
            frame_bgra = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
            frame_bgra[mask != 0] = [0, 0, 0, 0]
            cv2.imwrite(frame_path, frame_bgra)

    # Step 3: Overlay transparent frames onto background frames
    def overlay_frames(transparent_frames_path, background_frames_path, output_frames_path):
        transparent_frame_files = sorted(os.listdir(transparent_frames_path))
        background_frame_files = sorted(os.listdir(background_frames_path))

        # Align frame counts by trimming extra frames from the longer video
        min_frame_count = min(len(transparent_frame_files), len(background_frame_files))
        transparent_frame_files = transparent_frame_files[:min_frame_count]
        background_frame_files = background_frame_files[:min_frame_count]

        for i, frame_file in enumerate(transparent_frame_files):
            transparent_frame = cv2.imread(os.path.join(transparent_frames_path, frame_file), cv2.IMREAD_UNCHANGED)
            background_frame = cv2.imread(os.path.join(background_frames_path, background_frame_files[i]))

            if transparent_frame.shape[2] == 4:  # Check for alpha channel
                alpha_channel = transparent_frame[:, :, 3] / 255.0
                for c in range(3):  # Blend RGB channels
                    background_frame[:, :, c] = background_frame[:, :, c] * (1 - alpha_channel) + transparent_frame[:, :, c] * alpha_channel

            output_frame_path = os.path.join(output_frames_path, f'output_frame_{i:04d}.png')
            cv2.imwrite(output_frame_path, background_frame)

        print(f"Overlay complete. {min_frame_count} frames processed.")

    # Step 4: Combine frames into a video
    def create_video_from_frames(frames_path, output_video_path, fps=30):
        frame_files = sorted(os.listdir(frames_path))
        if not frame_files:
            raise ValueError("No frames found to create a video.")

        first_frame = cv2.imread(os.path.join(frames_path, frame_files[0]))
        height, width, _ = first_frame.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        for frame_file in frame_files:
            frame = cv2.imread(os.path.join(frames_path, frame_file))
            video_writer.write(frame)

        video_writer.release()
        print(f"Video saved at {output_video_path}.")

    # Execute the pipeline
    background_video_path = load_background_video()  # Load dynamic background video
    merged_frame_count = extract_frames_from_video(merged_video_path, merged_video_frames_path)
    replace_black_background_with_transparency(merged_video_frames_path)
    background_frame_count = extract_frames_from_video(background_video_path, background_frames_path)
    overlay_frames(merged_video_frames_path, background_frames_path, output_frames_path)
    create_video_from_frames(output_frames_path, final_output_video_path, fps=30)

    return age_message, "/content/drive/MyDrive/agebased_motion_application_deeplearning_project/final_outputvideo/final_output_video.mp4"


# UI Layout
# UI Layout
def create_ui():
    # Define UI elements
    with gr.Blocks(css=".gradio-container {background-color: black; color: white;}") as demo:
        # Title
        gr.Markdown("<h1 style='text-align: center; color: white;'>Generate Age-specific Animation using AI</h1>")

        # Display Image (Introductory image preview)
        gr.Image(
            value="/content/drive/MyDrive/agebased_motion_application_deeplearning_project/websiteimagesample.png",
            label="Animation Preview",
            interactive=False
        )

        # Upload Section
        gr.Markdown("<h2 style='text-align: center; color: white;'>Upload 3 Images!</h2>")

        # Image Upload Boxes with Preview
        with gr.Row():
            image1 = gr.Image(label="Image 1", interactive=True, type="filepath")
            image2 = gr.Image(label="Image 2", interactive=True, type="filepath")
            image3 = gr.Image(label="Image 3", interactive=True, type="filepath")

        # Generate Video Button
        generate_button = gr.Button("Generate Video")

        # Detected Ages Textbox
        output_ages = gr.Textbox(label="Detected Ages", interactive=False)

        # Playable Video Section
        output_video = gr.Video(label="Generated Video")



        # Button Click Action
        generate_button.click(
            fn=agespecificmotion,  # Call the external function
            inputs=[image1, image2, image3],  # Pass inputs to the function
            outputs=[output_ages, output_video]  # Display both ages and video
        )

    return demo

# Launch the UI
create_ui().launch()
